In [1]:
import json
from authorization import SpotifyAuth

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
with open('client-secrets-rnr.json','r+') as secrets_file:
    secrets = json.load(secrets_file)

client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
redirect_url = secrets["redirect_url"]

In [5]:
spotify_authorisation = SpotifyAuth(client_id=client_id, client_secret=client_secret, redirect_url=redirect_url)
spotify_authorisation.set_access_token_from_file('access_token.json')

# Get User Details

In [6]:
import spotify_functions as spotify

In [7]:
spotify.get_user_details(spotify_authorisation.get_access_token())

{'display_name': 'sdspot2034',
 'external_urls': {'spotify': 'https://open.spotify.com/user/sdspot2034'},
 'href': 'https://api.spotify.com/v1/users/sdspot2034',
 'id': 'sdspot2034',
 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b8280153af8b79b3144b24f1291',
   'height': 64,
   'width': 64},
  {'url': 'https://i.scdn.co/image/ab6775700000ee8580153af8b79b3144b24f1291',
   'height': 300,
   'width': 300}],
 'type': 'user',
 'uri': 'spotify:user:sdspot2034',
 'followers': {'href': None, 'total': 9},
 'country': 'IN',
 'product': 'premium',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'email': 'shreyandas2034@gmail.com'}

In [8]:
import datetime
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
# print(yesterday.timestamp()*1e3)
cdc_time = int(yesterday.timestamp()*1e3)

# Get All Songs Played in the last one day

In [9]:
token = spotify_authorisation.get_access_token()
history = spotify.get_listening_history(token, cdc_time)['items']

## EDA over recent history

### Number of songs played

In [10]:
len(history)

38

### Fetch details of a song

#### See available data for a song

In [11]:
history[0].keys()

dict_keys(['track', 'played_at', 'context'])

In [12]:
list(history[0]['track'].keys())

['album',
 'artists',
 'disc_number',
 'duration_ms',
 'explicit',
 'external_ids',
 'external_urls',
 'href',
 'id',
 'is_local',
 'name',
 'popularity',
 'preview_url',
 'track_number',
 'type',
 'uri']

In [13]:
history[0]['played_at']

'2024-08-08T16:58:09.773Z'

In [14]:
print(history[0]['context'])

None


# Transform

## Dimensions

### DIM_ARTIST

In [15]:
import pandas as pd
from sqlalchemy import Table, MetaData, Column, Integer, String
import sql_functions

In [16]:
# stacked list-comprehension
artist_list = [artist for artists in [play['track']['artists'] for play in history] for artist in artists]

In [24]:
df_artists = pd.DataFrame(artist_list)
df_artists = df_artists.drop(columns=['href','uri','external_urls'])
df_artists = df_artists.rename(columns={'id':'spotify_id','name':'artist_name','type':'artist_type'})
df_artists.drop_duplicates(inplace=True)
# df_artists

#### Load (Type 1 SCD)

In [25]:
metadata = MetaData()

# Define the dimension table schema
dim_table = Table('DIM_ARTIST', metadata,
    Column('artist_wid', Integer, primary_key=True, autoincrement=True),
    Column('spotify_id', String(255), nullable=False, unique=True),
    Column('artist_name', String(255)),
    Column('artist_type', String(255))
)

In [26]:
sql_functions.save_to_sql(df_artists, dim_table, key_cols=['spotify_id'], wid=['artist_wid'], mode='upsert')

### DIM_ALBUM

In [27]:
album_list = [play['track']['album'] for play in history]

In [28]:
def get_url(x,height=640):
    for elem in x:
        if elem['height'] == height: return elem['url']
    return None

In [29]:
df_album = pd.DataFrame(album_list)
df_album['cover_image_url'] = df_album['images'].apply(get_url)
df_album = df_album.drop(columns=['href','uri','external_urls', 'images','artists','type'])
df_album = df_album.drop_duplicates()
df_album = df_album.rename(columns={'id':'album_id','name':'album_name'})
# df_album

#### Load (Type 1 SCD)

In [30]:
from sqlalchemy import MetaData, Table, Column, Integer, String, Text, Date

metadata = MetaData()

dim_album = Table(
    'DIM_ALBUM', metadata,
    Column('album_wid', Integer, primary_key=True, autoincrement=True),
    Column('album_id', String(62), nullable=False, unique=True),
    Column('album_type', String(20), nullable=False),
    Column('album_name', Text, nullable=False),
    Column('release_date', Date),
    Column('release_date_precision', String(20)),
    Column('total_tracks', Integer),
    Column('cover_image_url', Text)
)

In [31]:
sql_functions.save_to_sql(df_album, dim_album, key_cols = ['album_id'], wid = ['album_wid'], mode = 'upsert')

### BRIDGE_ALBUM_ARTIST

In [33]:
dim_artist = sql_functions.read_from_sql('DIM_ARTIST')
dim_album = sql_functions.read_from_sql('DIM_ALBUM')
df_album = pd.DataFrame(album_list)
df_album_map = df_album[['id','artists']]
df_album_map = df_album_map.explode('artists')
df_album_map['artist_id'] = df_album_map['artists'].apply(lambda x:x['id'])
df_album_map = df_album_map.drop(columns=['artists'])
# df_album_map

In [37]:
# Join with DIM_ARTIST, DIM_ALBUM to get ARTIST_WID, ALBUM_WID
df_bridge_album_artist = df_album_map.merge(dim_artist, left_on='artist_id', right_on='spotify_id')
df_bridge_album_artist = df_bridge_album_artist.merge(dim_album, left_on='id', right_on='album_id')
df_bridge_album_artist = df_bridge_album_artist[['album_wid','artist_wid']]
df_bridge_album_artist = df_bridge_album_artist.drop_duplicates()
# df_bridge_album_artist

#### Load (Type 1 SCD)

In [38]:
from sqlalchemy import MetaData, Table, Column, Integer

metadata = MetaData()

bridge_album_artist = Table(
    'BRIDGE_ALBUM_ARTIST', metadata,
    Column('album_artist_map_wid', Integer, primary_key=True, autoincrement=True),
    Column('album_wid', Integer, nullable=False),
    Column('artist_wid', Integer, nullable=False)
)

In [39]:
sql_functions.save_to_sql(
    df_bridge_album_artist
    , bridge_album_artist
    , key_cols = ['album_wid', 'artist_wid']
    , wid = ['album_artist_map_wid']
    , mode = 'ignore'
)

### DIM_SONG

In [59]:
dim_album = sql_functions.read_from_sql('DIM_ALBUM')

In [60]:
history[0]['track'].keys()

dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [61]:
history[0]['track']

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6qqNVTkY8uBg9cP3Jd7DAH'},
    'href': 'https://api.spotify.com/v1/artists/6qqNVTkY8uBg9cP3Jd7DAH',
    'id': '6qqNVTkY8uBg9cP3Jd7DAH',
    'name': 'Billie Eilish',
    'type': 'artist',
    'uri': 'spotify:artist:6qqNVTkY8uBg9cP3Jd7DAH'}],
  'external_urls': {'spotify': 'https://open.spotify.com/album/3AafSrFIbJPH6BJHiJm1Cd'},
  'href': 'https://api.spotify.com/v1/albums/3AafSrFIbJPH6BJHiJm1Cd',
  'id': '3AafSrFIbJPH6BJHiJm1Cd',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273ed317ec13d3de9e01fb99c9e',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e02ed317ec13d3de9e01fb99c9e',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d00004851ed317ec13d3de9e01fb99c9e',
    'width': 64}],
  'name': 'What Was I Made For? [From The Motion Picture "Barbie"]',
  'release_date': '2023-07

In [62]:
track_list = [play['track'] for play in history]

In [66]:
df_songs = pd.DataFrame(track_list)
df_songs = df_songs.explode('artists')
df_songs['artist_id'] = df_songs['artists'].apply(lambda x: x['id'])
df_songs['isrc_id'] = df_songs['external_ids'].apply(lambda x: x['isrc'])
df_songs = df_songs.drop(columns=['href','uri','external_urls','artists','external_ids', 'artist_id', 'popularity'])
df_songs['album_id'] = df_songs['album'].apply(lambda x:x['id'])
df_songs = df_songs.merge(dim_album[['album_wid','album_id']], left_on='album_id', right_on='album_id')
df_songs = df_songs.drop(columns=['album','album_id'])
df_songs = df_songs.rename(columns={'id':'song_id'})
df_songs = df_songs.drop_duplicates()
df_songs

,disc_number,duration_ms,explicit,song_id,is_local,name,preview_url,track_number,type,isrc_id,album_wid
0,1,222369,False,6wf7Yu7cxBSPrRlWeSeK0Q,False,What Was I Made For? [From The Motion Picture ...,https://p.scdn.co/mp3-preview/e58c595e821ffdbd...,1,track,USUM72307484,34
6,1,236413,False,1R0a2iXumgCiFb7HEZ7gUE,False,Don’t Blame Me,https://p.scdn.co/mp3-preview/8edf06fbc7550b9c...,4,track,USCJY1750006,35
7,1,197960,False,5ZrDlcxIDZyjOzHdYW1ydr,False,Barbie Girl,https://p.scdn.co/mp3-preview/e50666a04ed8364e...,3,track,DKBKA9700403,36
8,1,109750,True,741UUVE2kuITl0c6zuqqbO,False,Barbie World (with Aqua) [From Barbie The Album],https://p.scdn.co/mp3-preview/f654928ccaa4e6a9...,1,track,USAT22306213,37
11,1,176579,False,1vYXt7VSjH9JIM5oRRo7vA,False,Dance The Night - From Barbie The Album,https://p.scdn.co/mp3-preview/acaea048f50a3b30...,1,track,USAT22305457,38
12,1,100506,False,0QI75NHBAVISJRrlsujmP7,False,薄暮,https://p.scdn.co/mp3-preview/78c74dac476df6fc...,14,track,JPE300702474,39
13,1,200185,False,0u2P5u6lvoDfwTYjAADbn4,False,lovely (with Khalid),https://p.scdn.co/mp3-preview/18b3cbbad11e488c...,1,track,USUM71804190,40
15,1,334743,False,6mFkJmJqdDVQ1REhVfGgd1,False,Wish You Were Here,https://p.scdn.co/mp3-preview/e3d046771206da91...,4,track,GBN9Y1100088,41
16,1,235493,False,2ENexcMEMsYk0rVJigVD3i,False,Only Girl (In The World),https://p.scdn.co/mp3-preview/ee800fcd3f6b738d...,5,track,USUM71023200,42
17,1,187520,False,2nLtzopw4rPReszdYBJU6h,False,Numb,https://p.scdn.co/mp3-preview/d6d67b4f23b4865e...,13,track,USWB10300474,43


#### Load (Type 1 SCD)

In [67]:
from sqlalchemy import BigInteger, Boolean, Text


metadata = MetaData()

dim_song = Table(
    'DIM_SONG', metadata,
    Column('song_wid', Integer, primary_key=True, autoincrement=True),
    Column('song_id', String(62), nullable=False, unique=True),
    Column('album_wid', Integer),
    Column('disc_number', Integer),
    Column('duration_ms', BigInteger, nullable=False),
    Column('explicit', Boolean),
    Column('is_local', Boolean),
    Column('name', Text, nullable=False),
    Column('preview_url', Text),
    Column('track_number', Integer),
    Column('type', String(50)),
    Column('isrc_id', String(100))
)

In [68]:
sql_functions.save_to_sql(df_songs, dim_song, key_cols = ['song_id'], wid = ['song_wid'], mode = 'upsert')